# Introduction

Players engage with video games in many different ways. Some log in only for short sessions, while others spend much more time exploring and return more frequently. Beyond in-game activities, players also differ in how closely they stay connected to the game’s community. One way to stay engaged is by subscribing to the game’s newsletter for updates and upcoming events. Because subscription rates often reflect higher interest and community involvement, it’s useful to identify which types of players are more likely to subscribe. 

The two data sets: players.csv and sessions.csv center on real data collected from the UBC Computer Science research group studying player behaviour on a Minecraft server. The goal for the research is to better target recruitment and ensure they have adequate server resources. 

To support the research group’s recruitment efforts, our project aims to answer the question: Can the played hours, and age of a player predict whether or not they subscribed to the newsletter in the players dataset? This helps determine which factors are most associated with engagement so the group can better tailor their efforts accordingly.

We will only be working with players.csv, which includes data about all unique players. It contains information such as name, age, played hours, subscription status, and more. A detailed description of the dataset is provided below:



The data set consists of 196 observations and 7 variables
|Variable Name|Data Type|Description|
|------------------|----------------------------|-------------|
| experience | Character|experience level of the players at the game from beginner to regular to amateur to veteran to pro |
| subscribe| Logical|if the players are subscribed to the newsletter |
| hashed_email|Character|  unique encrypted way to identify each player |
| played_hours| Numeric|total hours the player has played the game |
| name| Character|player's name|
| gender| Character| player's gender, either Male, Female, Other, Two Spirited, or Prefer not to say |
| Age | Numeric| players age in years| 




# Questions

### *Broad Question:*

What player characteristics and behaviours are most predictive of subscribing to a game-related newsletter, and how do these features differ between various player types?

### *Specific Question:*

 Can the played hours, and age of a player predict whether or not they subscribed to the newsletter in the players dataset?


# Methods (Narration)

We start by loading the R libraries we need for this analysis. 

**Data Preparation**

First, we import the players.csv dataset and clean it up. Variables like gender, experience level, and subscription status are converted to factors. The subscription variable originally uses TRUE/FALSE, so we recorded it to “Yes” and “No” for clarity. Player names are removed as it’s not necessary for our analysis. We’re looking at a general trend rather than specific data for each player. Any rows with missing values are also dropped to make sure the modeling runs smoothly.

**Exploratory Data Analysis**

To get a quick sense for the data, we created a few plots:

- A histogram showing the relationship between subscription status and hours played on the server.
- A histogram showing the age distribution of subscribers versus non-subscribers.
- A scatterplot of age versus hours played to see if certain age and playtime combinations are linked to subscription.

These visualizations give a quick overview of how the predictors relate to subscription and help us see what patterns might exist before modeling.

**Modeling Approach**

We use k-nearest neighbors (KNN) classification to predict a player’s subscription status using Age and played_hours as predictors and subscribe as the response variable. We do not include experience level or gender because KNN requires numerical predictors.

KNN is appropriate here because the response variable is categorical, and the predictors are numerical. KNN can capture non-linear relationships and is flexible. We assume the data is tidy and that all predictors contribute equally, which is why we standardize the data using centering and scaling.

The data is split into a training set (75%) and a test set (25%) while keeping the same proportion of subscribers in each set. Standardization is applied only to the training set using step_center and step_scale.

We tune the number of neighbors k from 1 to 30 and use 5-fold cross-validation on the training data to identify the most accurate value of k. 

After selecting the best k, the code trains a final KNN model on the full training set and then evaluates it on the test data. This allows us to see how well age and playtime can actually predict which players are more engaged and likely to subscribe.


# Methods (Code)

In [ ]:
library(tidyverse)
library(repr)
library(tidymodels)
library(GGally)
library(ISLR)
library(purrr)
library(lubridate)
options(repr.matrix.max.rows = 6)


In [ ]:
players<-read_csv("https://raw.githubusercontent.com/RadinAlikhani/dsci100-solo-project/refs/heads/main/players.csv")|>
    mutate(gender=as_factor(gender),experience=as_factor(experience),subscribe=as_factor(subscribe))|>
    mutate(subscribe=fct_recode(subscribe, "Yes" = "TRUE", "No" = "FALSE"))|>
    select(-name)

players


In [ ]:
played_time_plot<-players|>
    ggplot(aes(x=played_hours,fill=subscribe))+
    geom_histogram()+
    labs(x="Time Played (Hours)",y="Number of Players",fill="Subscribed?")+
    ggtitle("The distribution of playtime of players and subscription status")
played_time_plot

age_plot<-players|> #change this to a boxplot of age by subscription, or proportion of subscribers within age groups
    ggplot(aes(x=Age,fill=subscribe))+
    geom_histogram()+
    labs(x="Age of player (Years)",y="Number of Players",fill="Subscribed?")+
    ggtitle("The distribution of age of players and subscription status")
age_plot

age_and_time_plot<-players|>
    ggplot(aes(x=Age,y=played_hours,color=subscribe))+
    geom_point()+
    labs(x="Age of Players (Years)",y="Time Played (hours)",color="Subscribed?")+
    ggtitle("The relationship of age and time played of a player with subscription status")
age_and_time_plot


## Observations from these plots
#### Plot 1:
- players with more than 25 hours are usually all subscribed.
- helps distinguish between the different hours and subscription status
- can be used as predictor

#### Plot 2
- ages from 0 to 20 (besides 19) had higher proportion of subscribers to non subscribers compared to ages after 20, especially after 30
- can be used as predictor.

#### Plot 3
- Players in the age range of 10-20 have more hours on average compared to higher ages


In [ ]:
players<-drop_na(players)

players_split<-initial_split(players,prop=0.75,strata=subscribe)
players_train<-training(players_split)
players_test<-testing(players_split)

players_recipe<-recipe(subscribe~played_hours+Age,data=players_train)|>
step_scale(all_predictors())|>
step_center(all_predictors())

players_vfold<-vfold_cv(players_train,v=5,strata=subscribe)

knn_spec<-nearest_neighbor(weight_func="rectangular",neighbors=tune())|>
set_engine("kknn")|>
set_mode("classification")

k_vals<-tibble(neighbors=seq(from=1,to=30,by=1))
               
knn_results <- workflow() |>
  add_recipe(players_recipe) |>
  add_model(knn_spec) |>
  tune_grid(resamples = players_vfold, grid = k_vals) |>
  collect_metrics()

accuracies <- knn_results |>
  filter(.metric == "accuracy")

best_k <- accuracies |>
        arrange(desc(mean)) |>
        head(1) |>
        pull(neighbors)
best_k

best_knn<-knn_spec<-nearest_neighbor(weight_func="rectangular",neighbors=best_k)|>
set_engine("kknn")|>
set_mode("classification")

knn_fit<-workflow()|>
add_recipe(players_recipe)|>
add_model(best_knn)|>
fit(data=players_train)
knn_fit

players_test_predictions <- predict(knn_fit, players_test) |>
  bind_cols(players_test)

players_test_predictions |>
  metrics(truth = subscribe, estimate = .pred_class) |>
  filter(.metric == "accuracy")


                                      
                                      
    
                                      